# SGEN_Py Quick Start tutorial

## About
SGEN_Py (Stochastic Gene Expression in Neurons) is a Python module that allows computing the first and second moments of active gene, mRNA and protein counts everywhere in arbitrary neurons. It does so by solving a linear stochastic model of gene->mRNA->protein dynamics. The module can also be used to run a Monte Carlo simulation of the underlying stochastic process, which can be used as a generative model of protein noise. However, the real strength of this software is its ability to obtain expectations and correlations of all variables in the model without expensive Monte Carlo simulation.

In this tutorial, we will:
- Manually construct a simple neuron
- Compute stationary expectations and correlations of all variables: active gene counts, mRNA counts and protein counts
- Visualise the computed distributions
- Run the Gillespie algorithm on this simple neuron
- Load a neuron from .swc file
- Visualise the neuron and the computed mRNA/protein distributions


## Manual neuron creation
#### Importing dependencies

In [1]:
import sys
sys.path.append('../')

import SGEN_Py as sg
import numpy as np

### "Growing" a neuron
Below, we create a Y-shaped neuron consisting of a cylindrical soma with a radius and length of 20$\mu m$ and a dendrite with a single fork, with every dendritic branch having a length of 200$\mu m$.
#### Creating a soma
In SGEN a neuron is a tree of compartments starting from the soma. 

In [2]:
# Create a soma with desired parameters and location
soma = sg.Soma("soma", length=20, x=0, y=0, z=0, radius=20, protein_forward_trafficking_velocity=.001)

#### Creating a primary dendritic branch
When creating a compartment other than soma, we must pass its parent (i.e., the compartment it attaches to) to its constructor. A dendritic branch (a piece of dendrite without branching) can be represented as a list of dendritic segments used for discretisation. (The larger the number of dendritic segments, the closer the model is to a continuum, but the harder it is to compute.) Let's initialise the primary (directly attached to the soma) dendritic branch by attaching a dendritic segment to the soma.

In [3]:
# Define the length of a single dendritic branch
Dendrite_length = 200 #um
# Define the number of cylindrical segments to discretise dendritic branches
N_dendritic_segments = 15

In [4]:
# Initialise a primary dendritic branch
primary_branch = [sg.Dendritic_segment(parent=soma,
                                       name = "d_1-1",
                                       length = Dendrite_length/N_dendritic_segments)]

In the above code, we created a list containing one dendritic segment with a certain name and length attached to the soma. See help for all possible constructor arguments including rates of mRNA/protein dynamics.

Now let's create more dendritic segments to obtain the entire primary dendritic branch of length 200$\mu m$.

In [5]:
# Adding dendritic segments to the primary dendritic branch
for i in np.arange(1,N_dendritic_segments):
    primary_branch.append(sg.Dendritic_segment(parent=primary_branch[-1],
                                               name=f"d_1-{i+1}",
                                               length=Dendrite_length/N_dendritic_segments))

In the above code, we created the primary dendritic branch as a list of short dendritic segments, where every next segment attaches to the previous one.

Now let's fork the dendrite and create two secondary dendritic branches attaching them to the last segment of the primary branch created above.

In [6]:
# Initialise 1st secondary dendritic branch
secondary_branch_1 = [sg.Dendritic_segment(parent=primary_branch[-1],
                                           name="d_1_1-1",
                                           length=Dendrite_length/N_dendritic_segments,
                                           d_theta=30*np.pi/360,
                                           d_phi=0)]
# Adding dendritic segments to the 1st secondary dendritic branch
for i in np.arange(1,N_dendritic_segments):
    secondary_branch_1.append(sg.Dendritic_segment(parent=secondary_branch_1[-1],
                                                   name=f"d_1_1-{i+1}",
                                                   length=Dendrite_length/N_dendritic_segments))

# Initialise 2nd secondary dendritic branch
secondary_branch_2 = [sg.Dendritic_segment(parent=primary_branch[-1],
                                           name="d_1_1-1",
                                           length=Dendrite_length/N_dendritic_segments,
                                           d_theta=-30*np.pi/360,
                                           d_phi=0)]
# Adding dendritic segments to the 2nd secondary dendritic branch
for i in np.arange(1,N_dendritic_segments):
    secondary_branch_2.append(sg.Dendritic_segment(parent=secondary_branch_2[-1],
                                                   name=f"d_1_1-{i+1}",
                                                   length=Dendrite_length/N_dendritic_segments,
                                                   radius=5*np.exp(-1/50*i)))

Note that we specified the parameters d_theta and d_phi, corresponding to the difference in the spherical coordinates ($\theta$ and $\phi$) compared to the parent compartment. These angular differences are set to zero by default, so if they are not specified, the dendrite grows in a straight line. 

Now let's "grow" a dendritic spine on our primary dendrite at roughly 1/3 of its length.

In [7]:
# Creating a dendritic spine on the primary branch
s_1_1 = sg.Spine(parent=primary_branch[N_dendritic_segments//3],
                 name="s_1_1",
                 length=10,
                 radius=1)

Now let's add five more spines at different locations and on different branches.

In [8]:
# Adding more spines, some of which grow in opposite direction (d_theta=-np.pi/2)
s_1_2 = sg.Spine(parent=primary_branch[2*N_dendritic_segments//3],
                 name="s_1_2",
                 length=10,
                 radius=1,
                 d_theta=-np.pi/2)
s_11_1 = sg.Spine(parent=secondary_branch_1[N_dendritic_segments//3],
                  name = "s_11_1",
                  length=10,
                  radius=1)
s_11_2 = sg.Spine(parent=secondary_branch_1[2*N_dendritic_segments//3],
                  name = "s_11_2",
                  length=10,
                  radius=1,
                  d_theta=-np.pi/2)
s_12_1 = sg.Spine(parent=secondary_branch_2[N_dendritic_segments//3],
                  name="s_12_1",
                  length=10,
                  radius=1,
                  d_theta=-np.pi/2)
s_12_2 = sg.Spine(parent=secondary_branch_2[2*N_dendritic_segments//3],
                  name="s_12_2",
                  length=10,
                  radius=1)

### Initialising a neuron
Now that the tree of compartments is ready, we can initialise the instance of class Neuron that provides the main functionality of the library. For that, we pass the soma to the constructor of the Neuron class.

In [9]:
neuron = sg.Neuron(soma, "Test_neuron")

### Visualisation

Now let's visualise our neuron.

In [10]:
neuron.draw_3d()

## Stationary gene-mRNA-protein moments

The expectations of all variables in the model can be computed as follows.

In [11]:
# Computing expectations of all variables in the model
expectations = neuron.expected_counts(dict_return=True)
print(expectations)

Expectations is a dictionary with the following keys.

In [12]:
print(expectations.keys())

If run with dict_return=True, neuron.expected_counts(dict_return=True) returns dictionaries for 'mRNA' and 'prot', whose keys correspond to compartment names. (Since all genes reside in the soma, there is no need to return the dictionary for the expected number of active genes, which can simply be accessed by the 'gene' key.) So the expected mRNA/protein counts in different compartments can be obtained as follows.

In [13]:
print('Expected number of active genes:', expectations['gene'])
print('Expected mRNA counts in the soma:', expectations['mRNA']['soma'])
print('Expected mRNA counts in d_1_1-3:', expectations['mRNA']['d_1_1-3'])
print('Expected protein counts in d_1_1-5:', expectations['prot']['d_1_1-5'])

#### Visualisation of concentrations
Expected concentrations are obtained by dividing the expected molecule counts by the volumes of the corresponding volumes as follows.

In [14]:
# Computing protein concentrations
concentrations = neuron.expected_counts()['prot']/neuron.volumes()
print(neuron.volumes())
print(concentrations)

Note that in the above code, we call neuron.expected_counts() with default dict_return=False, so that neuron.expected_counts()['prot'] returns a numpy array that can be divided by a numpy array of volumes of all compartments ordered in the same order, neuron.volumes().
Now we can visualise protein concentrations as follows.

In [15]:
neuron.draw_3d(concentrations)

### Stationary correlations
Now we compute stationary correlations (2nd moments) of all variables in the model defined as $\langle A\cdot B \rangle$ for variables $A$ and $B$, where $\langle ... \rangle$ stands for expectation (ensemble average).

In [16]:
correlations = neuron.correlations()

In our model, there are six types of correlations: gene-gene, gene-mRNA, mRNA-mRNA, gene-protein, mRNA-protein and protein-protein. Therefore, neuron.correlations() returns stationary correlations as a dictionary with the following keys.

Protein-protein correlators turn out to be computationally hardest to compute and the exact method involving matrix inversion scales as $\mathcal O(N^4)$ in memory consumption and $\mathcal O(N^6)$ in time, where $N$ is the number of compartments.

In [17]:
print(correlations.keys())

Let's look at some of the correlation matrices.

In [18]:
# gene-gene correlation is just a single number
print('Active gene count correlation:', correlations['gene-gene'])
# gene-mRNA and gene-protein correlations are vectors
print('Correlations of active gene count with mRNA counts:\n', correlations['gene-mRNA'])
# mRNA-mRNA, mRNA-protein and protein-protein correlations are matrices of different sizes
print('Correlation matrix of protein counts with protein counts:\n', correlations['prot-prot'])
# Note that mRNA-protein correlation matrix is not square in the presence of spines.
# This comes from the assumption that spines don't have mRNAs (which we may change). 
print('Shape of correlations of mRNA counts with protein counts:\n', correlations['mRNA-prot'].shape)

Now that the first and second moments of gene, mRNA and protein counts are computed and stored within the neuron, we can use them to compute variances, standard deviations and Pearson correlation coefficients as shown below.

In [19]:
# Examples of computing Pearson Correlation Coefficients
print('Pearson correlation coefficient of protein counts in spinces s_1_1 and s_12_2 = ', neuron.prot_prot_Pearson_correlation(s_1_1, s_12_2))
print('Pearson correlation coefficient of mRNA counts in the soma and protein counts in spine s_12_2 = ', neuron.mRNA_prot_Pearson_correlation(soma, s_12_2))
print('Pearson correlation coefficient of mRNA counts in the soma and the farthest dendritic segment = ', neuron.mRNA_prot_Pearson_correlation(soma, secondary_branch_2[-1]))

Therefore, if we want to compute the matrix of Pearson correlation coefficients of protein counts in all segments of the primary dendritic branch, we can do it as follows.

In [20]:
correlations = np.zeros((len(primary_branch), len(primary_branch)))
for i in range(len(primary_branch)):
    for j in range(i, len(primary_branch)):
        correlations[j,i] = correlations[i,j] = neuron.prot_prot_Pearson_correlation(primary_branch[i],primary_branch[j])
print("Pearson correlations coefficients of protein counts in primary dendrite:\n", correlations)

We can visualise the above computed PCC matrix using Matplotlib.

In [21]:
import matplotlib.pyplot as plt

# Add a color bar
im = plt.imshow(correlations)

cbar = plt.colorbar(im)
cbar.set_label("Pearson Correlation Coefficient")

# Add title and labels
plt.title("Pearson correlations for protein counts")
plt.xlabel("Compartment index on primary dendrite")
plt.ylabel("Compartment index on primary dendrite")

plt.tight_layout()
plt.show()

## Monte Carlo simulation
The above computations allow obtaining expectations and correlations of all variables in the model without resorting to stochastic simulation. However, sometimes it is useful to look at individual trajectories of the process, which can be obtained using the Gillespie algorithm as follows.


In [22]:
# sim_results = neuron.stationary_Gillespie_sim(record_times=range(0,3000,1), output_file_name='soma_prot_trafficking')
sim_results = neuron.load_Gillespie_sim('soma_prot_trafficking.csv')

Now let's visualise the dependence of protein counts in spine s_12_2 on time. (Change s_12_2 in compartment = s_12_2 to any other compartment e.g. soma or primary_branch[2] to see the corresponding results for that compartment.)

In [23]:
compartment = soma

plt.plot(sim_results['time'], sim_results[compartment.name() + '_prot'], color='blue', label='Gillespie trajectory')
plt.axhline(expectations['prot'][compartment.name()], color='cyan', alpha=.7, linewidth=2, label='Stationary expectation')

# Plotting stationary standard deviation
std = neuron.protein_standard_deviation(compartment)
plt.axhline(expectations['prot'][compartment.name()] + std, color='cyan', alpha=.7, linewidth=2, label='Stationary expectation' + r'$\pm\sigma$', linestyle='--')
plt.axhline(expectations['prot'][compartment.name()] - std, color='cyan', alpha=.7, linewidth=2, linestyle='--')

plt.xlim([0, max(sim_results['time'])])

plt.xlabel('Time in hours')
plt.ylabel('Protein count')

plt.legend()
plt.show()

To analyse correlations between different variables it is useful to plot them against each other. Below, we do this and compare the results with a bivariate Gaussian distribution with the exact mean and covariance matrix.

In [24]:
comp1, comp2 = s_1_1, s_12_2

plt.plot(sim_results[comp1.name()+'_prot'], sim_results[comp2.name()+'_prot'], color='blue', label='Gillespie trajectory', linestyle='', marker='.')
plt.plot(expectations['prot'][comp1.name()], expectations['prot'][comp2.name()], color='red', alpha=.7, label='Stationary expectation', marker='+', markersize=15, linestyle='')

# Creating bivariate Gaussian
mean = [expectations['prot'][comp1.name()], expectations['prot'][comp2.name()]]
std = [neuron.protein_standard_deviation(comp1), neuron.protein_standard_deviation(comp2)]

Sigma = np.array([[std[0]**2, neuron.prot_prot_covariance(s_12_1, s_12_2)],
                 [neuron.prot_prot_covariance(s_12_1, s_12_2), std[1]**2]])

# Create grid for PDF visualization
x = np.linspace(mean[0] - 3*std[0], mean[0] + 3*std[0], 100)
y = np.linspace(mean[1] - 3*std[1], mean[1] + 3*std[1], 100)
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))

# Compute PDF values
from scipy.stats import multivariate_normal
rv = multivariate_normal(mean, Sigma)
Z = rv.pdf(pos)

plt.contour(X, Y, Z, levels=5, alpha=.7, cmap='plasma')

plt.xlabel('Protein count')
plt.ylabel('Protein count')

plt.legend()
plt.show()

## Nonstationary moments and simulation

Above we looked at how to compute moments of the stationary distribution of all variables in the model and how to sample from such distribution using the Gillespie algorithm. However, we can simulate the behaviour of the model from an arbitrary initial condition and look at how our gene-mRNA-protein system reaches stationarity. For example, we can run

In [25]:
neuron.reset()

Now let's run the simulation and visualise the dependence of soma protein counts on time.

In [26]:
sim_results = neuron.Gillespie_sim(range(1000), output_file_name="nonstationary_Gillespie")

compartment = soma

plt.plot(sim_results['time'], sim_results[compartment.name() + '_prot'], color='blue', label='Gillespie trajectory')
plt.axhline(expectations['prot'][compartment.name()], color='cyan', alpha=.7, linewidth=2, label='Stationary expectation')

# Plotting stationary standard deviation
std = neuron.protein_standard_deviation(compartment)
plt.axhline(expectations['prot'][compartment.name()] + std, color='cyan', alpha=.7, linewidth=2, label='Stationary expectation' + r'$\pm\sigma$', linestyle='--')
plt.axhline(expectations['prot'][compartment.name()] - std, color='cyan', alpha=.7, linewidth=2, linestyle='--')

plt.xlim([0, max(sim_results['time'])])

plt.xlabel('Time in hours')
plt.ylabel('Protein count')

plt.legend()
plt.show()

Now, unlike for stationary_Gillespie_sim we used above, the output of the function reports "Number of burn-in events: 0" meaning that Gillespie_sim function started recording without waiting until the system reached stationarity. The unrecorded burn-in time can be set manually using the corresponding argument of the function as shown below.

In [27]:
new_sim_results = neuron.Gillespie_sim(range(100), burn_in=10)

compartment = soma

plt.plot(new_sim_results['time'], new_sim_results[compartment.name() + '_prot'], color='blue', label='Gillespie trajectory')
plt.axhline(expectations['prot'][compartment.name()], color='cyan', alpha=.7, linewidth=2, label='Stationary expectation')

# Plotting stationary standard deviation
std = neuron.protein_standard_deviation(compartment)
plt.axhline(expectations['prot'][compartment.name()] + std, color='cyan', alpha=.7, linewidth=2, label='Stationary expectation' + r'$\pm\sigma$', linestyle='--')
plt.axhline(expectations['prot'][compartment.name()] - std, color='cyan', alpha=.7, linewidth=2, linestyle='--')

plt.xlim([0, max(new_sim_results['time'])])

plt.xlabel('Time in hours')
plt.ylabel('Protein count')

plt.legend()
plt.show()

We can see that running the Gillespie_sim function does not reinitialise the system by default, but will reset all variables to zero if the reset argument is set to True.

### Nonstationary moments
Now let's look at how to compute the nonstationary moments of all variables without the computationally expensive Monte Carlo simulation. 


## Changing the parameters

## Working with neurons from .swc files

In [28]:
# Creating a new neuron specified in .swc file
# neuron = sg.Neuron("../data/morphologies/10_2REDO-850-GM18-Ctl-Ctl-Chow-BNL16A-CA1_Finished2e.swc")
swc_neuron = sg.Neuron("../data/morphologies/DD13-10-c6-2.CNG.swc")
# Computing expectations
swc_expectations = swc_neuron.expected_counts()
# Visualising
swc_neuron.draw_3d(swc_expectations['prot'])

In [29]:
sys.path.append('../build')
import _SGEN_Py as _sg

soma_  = sg.Soma(name='soma_')

_neuron = _sg._Neuron(soma_)

In [30]:
print(_neuron)

In [31]:
ae = _sg._Analytic_engine(_neuron)

In [32]:
ae.active_genes_expectation()
ae.stationary_mRNA_expectations()